<a href="https://colab.research.google.com/github/sharathchandra04/MLlearning/blob/main/classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import numpy as np

# Build the Perceptron Model
class Perceptron:

    def __init__(self, num_inputs, learning_rate=0.01):
        print(type(num_inputs))
        # Initialize the weight and learning rate
        self.weights = np.random.rand(num_inputs + 1)
        self.learning_rate = learning_rate

    # Define the first linear layer
    def linear(self, inputs):
        Z = inputs @ self.weights[1:].T + + self.weights[0]
        return Z

    # Define the Heaviside Step function.
    def Heaviside_step_fn(self, z):
        if z >= 0:
            return 1
        else:
            return 0

    # Define the Prediction
    def predict(self, inputs):
        Z = self.linear(inputs)
        try:
            pred = []
            for z in Z:
                pred.append(self.Heaviside_step_fn(z))
        except:
            return self.Heaviside_step_fn(Z)
        return pred

    # Define the Loss function
    def loss(self, prediction, target):
        loss = (prediction-target)
        return loss

    #Define training
    def train(self, inputs, target):
        prediction = self.predict(inputs)
        # print('prediction -> ', prediction)
        error = self.loss(prediction, target)
        # ******************  imp  ****************
        # this is basically (inputs*error)/100 if learning rate is 0.01
        # this is basically (inputs*error)/1000 if learning rate is 0.001
        # to completely use the potential of the learning-rate which is 0.001 run the train algorithm 1000 times
        # ******************  imp  ****************
        self.weights[1:] += self.learning_rate * error * inputs
        self.weights[0]  += self.learning_rate * error

    # Fit the model
    def fit(self, X, y, num_epochs):
        for epoch in range(num_epochs):
            for inputs, target in zip(X, y):
                self.train(inputs, target)

import numpy as np
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Generate a linearly separable dataset with two classes
X, y = make_blobs(n_samples=1000,
                  n_features=2,
                  centers=2,
                  cluster_std=3,
                  random_state=23)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=23,
                                                    shuffle=True
                                                   )

# Scale the input features to have zero mean and unit variance
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Set the random seed legacy
np.random.seed(23)

# Initialize the Perceptron with the appropriate number of inputs
print(X_train)
print('X_train.shape[1] --> ', X_train.shape[1])
perceptron = Perceptron(num_inputs=X_train.shape[1])

# Train the Perceptron on the training data
perceptron.fit(X_train, y_train, num_epochs=100)

# Prediction
pred = perceptron.predict(X_test)

# Test the accuracy of the trained Perceptron on the testing data
accuracy = np.mean(pred != y_test)
print("Accuracy:", accuracy)

# Plot the dataset
plt.scatter(X_test[:, 0], X_test[:, 1], c=pred)
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.show()


Streaming output truncated to the last 5000 lines.
prediction ->  1
prediction ->  0
prediction ->  1
prediction ->  1
prediction ->  0
prediction ->  1
prediction ->  0
prediction ->  0
prediction ->  1
prediction ->  0
prediction ->  0
prediction ->  0
prediction ->  0
prediction ->  1
prediction ->  0
prediction ->  0
prediction ->  0
prediction ->  0
prediction ->  0
prediction ->  1
prediction ->  1
prediction ->  1
prediction ->  0
prediction ->  0
prediction ->  1
prediction ->  1
prediction ->  1
prediction ->  0
prediction ->  1
prediction ->  0
prediction ->  0
prediction ->  0
prediction ->  0
prediction ->  0
prediction ->  0
prediction ->  1
prediction ->  0
prediction ->  1
prediction ->  0
prediction ->  1
prediction ->  0
prediction ->  1
prediction ->  1
prediction ->  1
prediction ->  0
prediction ->  0
prediction ->  0
prediction ->  0
prediction ->  0
prediction ->  1
prediction ->  0
prediction ->  0
prediction ->  0
prediction ->  1
prediction ->  1
prediction -> 

KeyboardInterrupt: ignored

In [19]:
import numpy as np
num_inputs1 = [1, 2, 3, 4]
print(np.random.rand(2 + 1))

J = 2.03
Jnp = np.float64(J)
print(type(Jnp))
a = 5
b = 6
c = -5
d = a + + b
print(d)
print(a,b, c)
e = a + + c
print(e)
print(a,b, c)



[0.88494754 0.30040969 0.58958187]
<class 'numpy.float64'>
11
5 6 -5
0
5 6 -5
